In [13]:
import os, glob, shutil
import numpy as np
import pandas as pd
import torch
import torchaudio

PATH = "/Users/yijisuk/Documents/MAIN DRIVE/Local Development/audio-sentiment-analysis"
DATA_PATH = os.path.join(PATH, 'dataset')
NEW_DATA_PATH = os.path.join(PATH, 'approach-2', 'categorized_dataset')

In [14]:
audio_filepaths = [filename for filename in glob.glob(os.path.join(DATA_PATH, '*.wav'))]
audio_filenames = [filepath.split('/')[-1] for filepath in audio_filepaths]

emotion_categories = list(np.unique([filename.split('.')[0].split('_')[-1] for filename in audio_filenames]))
emotion_categories.remove('fear')

sentiment_categories = ['positive', 'neutral', 'negative']

nofear_audio_filenames = [filename for filename in audio_filenames if filename.split('/')[-1].split('.')[0].split('_')[-1] != 'fear']
nofear_audio_filepaths_dict = {key: [] for key in sentiment_categories}
nofear_audio_filepaths = [os.path.join(DATA_PATH, filename) for filename in nofear_audio_filenames]

In [15]:
emotion_categories

['angry', 'disgust', 'happy', 'neutral', 'ps', 'sad']

In [17]:
for filepath in nofear_audio_filepaths:
    emotion = filepath.split('/')[-1].split('.')[0].split('_')[-1]
    
    if emotion in ['happy', 'ps']:
        nofear_audio_filepaths_dict['positive'].append(filepath)
    elif emotion == 'neutral':
        nofear_audio_filepaths_dict['neutral'].append(filepath)
    else:
        nofear_audio_filepaths_dict['negative'].append(filepath)

for key in sentiment_categories:
    try:
        os.mkdir(os.path.join(NEW_DATA_PATH, key))
    except FileExistsError:
        continue

for key in nofear_audio_filepaths_dict:
    paths = nofear_audio_filepaths_dict[key]
    for path in paths:
        shutil.copy(path, os.path.join(NEW_DATA_PATH, key))

In [25]:
annot_file = pd.DataFrame(columns=['filename', 'label'])
annot_file['filename'] = nofear_audio_filenames

string_labels = [name.split('/')[-1].split('.')[0].split('_')[-1] for name in nofear_audio_filenames]
al_cat = {'positive': 0, 'neutral': 1, 'negative': 2}

annot_labels = []
for label in string_labels:
    if label in ['happy', 'ps']:
        annot_labels.append(al_cat['positive'])
    elif label == 'neutral':
        annot_labels.append(al_cat['neutral'])
    else:
        annot_labels.append(al_cat['negative'])

annot_file['label'] = annot_labels

In [26]:
annot_file.head()

,filename,label
0,YAF_wire_happy.wav,0
1,OAF_fat_disgust.wav,2
2,OAF_bought_happy.wav,0
3,OAF_fit_neutral.wav,1
4,OAF_join_neutral.wav,1


In [27]:
ANNOT_FILE_PATH = os.path.join(NEW_DATA_PATH, 'annot_file.csv')
annot_file.to_csv(ANNOT_FILE_PATH, index=False)